# Preparation for Group Assignment

This tutorial contains various small guides for tasks you will need or come in handy in the upcoming group assignment.

We're going to need a couple of packages for this tutorial:

In [ ]:
from atlite.gis import ExclusionContainer
from atlite.gis import shape_availability
import atlite
import rasterio as rio
from rasterio.plot import show
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt
import country_converter as coco
import atlite

## Preparatory Downloads

For this tutorial, we also need to download a few files, for which one can use the `urllib` library:

In [ ]:
from urllib.request import urlretrieve

In [ ]:
fn = "era5-2013-NL.nc"
url = "https://tubcloud.tu-berlin.de/s/bAJj9xmN5ZLZQZJ/download/" + fn
urlretrieve(url, fn)

In [ ]:
fn = "PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326-NL.tif"
url = f"https://tubcloud.tu-berlin.de/s/567ckizz2Y6RLQq/download?path=%2Fcopernicus-glc&files={fn}"
urlretrieve(url, fn)

In [ ]:
fn = "WDPA_Oct2022_Public_shp-NLD.tif"
url = (
    f"https://tubcloud.tu-berlin.de/s/567ckizz2Y6RLQq/download?path=%2Fwdpa&files={fn}"
)
urlretrieve(url, fn)

In [ ]:
fn = "GEBCO_2014_2D-NL.nc"
url = (
    f"https://tubcloud.tu-berlin.de/s/567ckizz2Y6RLQq/download?path=%2Fgebco&files={fn}"
)
urlretrieve(url, fn)

## Downloading historical weather data from ERA5 with `atlite`

First, let's load some small example country. Let's say, the Netherlands.

In [ ]:
fn = "https://tubcloud.tu-berlin.de/s/567ckizz2Y6RLQq/download?path=%2Fgadm&files=gadm_410-levels-ADM_1-NLD.gpkg"
regions = gpd.read_file(fn)

In [ ]:
regions.plot()

In this example we download historical weather data [ERA5 data](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview) on-demand for a cutout we want to create.

:::{note}
For this example to work, you should have
- installed the Copernicus Climate Data Store `cdsapi` package (`conda list cdsapi` or `pip install cdsapi`) and
- registered and setup your CDS API key as described on [this website](https://cds.climate.copernicus.eu/api-how-to). Note that there are different instructions depending on your operating system.
:::

A **cutout** is the basis for any of your work and calculations in `atlite`.

The cutout is created in the directory and file specified by the relative path. If a cutout at the given location already exists, then this command will simply load the cutout again. If the cutout does not yet exist, it will specify the new cutout to be created.

For creating the cutout, you need to specify the dataset (e.g. ERA5), a time period and the spatial extent (in latitude and longitude).

In [ ]:
minx, miny, maxx, maxy = regions.total_bounds
buffer = 0.25

In [ ]:
cutout = atlite.Cutout(
    path="era5-2013-NL.nc",
    module="era5",
    x=slice(minx - buffer, maxx + buffer),
    y=slice(miny - buffer, maxy + buffer),
    time="2013",
)

Calling the function `cutout.prepare()` initiates the download and processing of the weather data.
Because the download needs to be provided by the CDS servers, this might take a while depending on the amount of data requested.

:::{note}
You can check the status of your request [here](https://cds.climate.copernicus.eu/cdsapp#!/yourrequests).
:::

In [ ]:
cutout.prepare()

The data is accessible in `cutout.data`. Included weather variables are listed in `cutout.prepared_features`. Querying the `cutout` gives us some basic information on which data is contained in it.

In [ ]:
cutout.data

In [ ]:
cutout.prepared_features

In [ ]:
cutout

## Repetition: From Land Eligibility Analysis to Availability Matrix

We're going to use the plotting functions from previous exercises:

In [ ]:
def plot_area(masked, transform, shape):
    fig, ax = plt.subplots(figsize=(5, 5))
    ax = show(masked, transform=transform, cmap="Greens", vmin=0, ax=ax)
    shape.plot(ax=ax, edgecolor="k", color="None", linewidth=1)

First, we collect all exclusion and inclusion criteria in an `ExclusionContainer`.

In [ ]:
excluder = ExclusionContainer(crs=3035, res=300)
fn = "PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326-NL.tif"
excluder.add_raster(fn, codes=[50], buffer=1000, crs=4326)
excluder.add_raster(fn, codes=[20, 30, 40, 60], crs=4326, invert=True)
fn = "WDPA_Oct2022_Public_shp-NLD.tif"
excluder.add_raster(fn, crs=3035)
fn = "GEBCO_2014_2D-NL.nc"
excluder.add_raster(fn, codes=lambda x: x > 10, crs=4326, invert=True)

Then, we can calculate the available areas...

In [ ]:
masked, transform = shape_availability(regions.to_crs(3035).geometry, excluder)

... and plot it:

In [ ]:
plot_area(masked, transform, regions.to_crs(3035).geometry)

### Availability Matrix

In [ ]:
regions.index = regions.GID_1

In [ ]:
cutout = atlite.Cutout("era5-2013-NL.nc")

In [ ]:
?cutout.availabilitymatrix

In [ ]:
A = cutout.availabilitymatrix(regions, excluder)

In [ ]:
cap_per_sqkm = 1.7  # MW/km2

area = cutout.grid.set_index(["y", "x"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix = A.stack(spatial=["y", "x"]) * area * cap_per_sqkm

### Solar PV Profiles

In [ ]:
pv = cutout.pv(
    panel=atlite.solarpanels.CdTe,
    matrix=capacity_matrix,
    orientation="latitude_optimal",
    index=regions.index,
    per_unit=True,
)

In [ ]:
pv.to_pandas().head()

In [ ]:
pv.to_pandas().iloc[:, 0].plot()

### Wind Profiles

In [ ]:
wind = cutout.wind(
    atlite.windturbines.Vestas_V112_3MW,
    matrix=capacity_matrix,
    index=regions.index,
    per_unit=True,
)

In [ ]:
wind.to_pandas().head()

In [ ]:
wind.to_pandas().iloc[:, 0].plot()

## Merging Shapes in `geopandas`

Spatial data is often more granular than we need. For example, we might have data on sub-national units, but we’re actually interested in studying patterns at the level of countries.

Whereas in `pandas` we would use the `groupby()` function to aggregate entries, in `geopandas`, we aggregate geometric features using the `dissolve()` function.

Suppose we are interested in studying continents, but we only have country-level data like the country dataset included in `geopandas`. We can easily convert this to a continent-level dataset.

:::{note}
See also https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html
:::

In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

In [ ]:
world.head(3)

In [ ]:
continents = world.dissolve(by="continent").geometry

continents.plot();

If we are interested in the population per continent, we can pass different aggregation strategies to the `dissolve()` functionusing the `aggfunc` argument:

https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html#dissolve-arguments

In [ ]:
continents = world.dissolve(by="continent", aggfunc=dict(pop_est="sum"))

In [ ]:
continents.plot(column="pop_est");

You can also pass a `pandas.Series` to the `dissolve()` function that describes your mapping for more exotic aggregation strategies (e.g. by first letter of the country name):

In [ ]:
world.dissolve(by=world.name.str[0], aggfunc=dict(pop_est="sum")).plot(column="pop_est")

## Representative Points and Crow-Fly Distances

The following example includes code to retrieve representative points from polygons and to calculate the distance on a sphere between two points.

:::{note}
See also https://en.wikipedia.org/wiki/Haversine_formula

See also https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.distance.html
:::

In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

In [ ]:
points = world.representative_point()

In [ ]:
fig, ax = plt.subplots()
world.plot(ax=ax)
points.plot(ax=ax, color="red", markersize=3);

In [ ]:
points = points.to_crs(4087)
points.index = world.iso_a3

In [ ]:
distances = pd.concat({k: points.distance(p) for k, p in points.items()}, axis=1).div(
    1e3
)  # km

In [ ]:
distances.loc["DEU", "NLD"]

In [ ]:
cutout.data.temperature.sel(time="2013-01-01 00:00").to_pandas().head()

## Global Equal-Area and Equal-Distance CRS

Previously, we used EPSG:3035 as projection to calculate the area of regions in km². However, this projection is not correct for regions outside of Europe, so that we need to pick different, more suitable projections for calculating areas and distances between regions.

- **for calculating distances:** [WGS 84 / World Equidistant Cylindrical (EPSG:4087)](https://epsg.io/4087)

- **for calculating areas:** [Mollweide (ESRI:54009)](https://epsg.io/54009)

The unit of measurement for both projections is metres.

In [ ]:
AREA_CRS = "ESRI:54009"
DISTANCE_CRS = "EPSG:4087"

In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

In [ ]:
world.to_crs(AREA_CRS).plot()

In [ ]:
world.to_crs(DISTANCE_CRS).plot()

## Country Converter

> The country converter (coco) is a Python package to convert and match country names between different classifications and between different naming versions. 

:::{note}
See also https://github.com/konstantinstadler/country_converter
:::

In [ ]:
import country_converter as coco

Convert various country names to some standard names, specifying source and target classification scheme:

In [ ]:
coco.convert(names="NLD", to="name_official")

In [ ]:
coco.convert(names="NLD", to="iso2")

In [ ]:
coco.convert(names="NLD", src="iso3", to="iso2")

In [ ]:
country_list = ["AE", "AL", "AM", "AO", "AR"]

In [ ]:
coco.convert(names=country_list, src="iso2", to="short_name")

List of included country classification schemes:

In [ ]:
cc = coco.CountryConverter()
cc.valid_country_classifications

## Gurobi

[Gurobi](https://www.gurobi.com/) is one of the most powerful solvers to solve optimisation problems. 
It is a commercial solver, with free academic licenses.

:::{note}
Using this solver for the group assignment is optional. You can also use other open-source alternatives, but they might just take a little longer to solve.
:::

To set up Gurobi, you need to:

- Register at https://pages.gurobi.com/registration/ with your institutional e-mail address (e.g. `@campus.tu-berlin.de`).
- Follow the getting started guide for your respective operating system at https://www.gurobi.com/documentation/quickstart.html (this includes a guide for retrieving your academic license and installing the software).
- In your `conda` environment, install `gurobipy` with `pip install gurobipy`.
